# Task 7: AutoFeatureSelector Tool
## This task is to test your understanding of various Feature Selection methods outlined in the lecture and the ability to apply this knowledge in a real-world dataset to select best features and also to build an automated feature selection tool as your toolkit

### Use your knowledge of different feature selector methods to build an Automatic Feature Selection tool
- Pearson Correlation
- Chi-Square
- RFE
- Embedded
- Tree (Random Forest)
- Tree (Light GBM)

### Dataset: FIFA 19 Player Skills
#### Attributes: FIFA 2019 players attributes like Age, Nationality, Overall, Potential, Club, Value, Wage, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from collections import Counter
import math
from scipy import stats

In [4]:
player_df = pd.read_csv("fifa19.csv")

In [5]:
type(player_df)

pandas.core.frame.DataFrame

In [6]:
numcols = ['Overall', 'Crossing','Finishing',  'ShortPassing',  'Dribbling','LongPassing', 'BallControl', 'Acceleration','SprintSpeed', 'Agility',  'Stamina','Volleys','FKAccuracy','Reactions','Balance','ShotPower','Strength','LongShots','Aggression','Interceptions']
catcols = ['Preferred Foot','Position','Body Type','Nationality','Weak Foot']

In [7]:
player_df = player_df[numcols+catcols]

In [8]:
player_df

,Overall,Crossing,Finishing,ShortPassing,Dribbling,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,...,ShotPower,Strength,LongShots,Aggression,Interceptions,Preferred Foot,Position,Body Type,Nationality,Weak Foot
0,94,84.0,95.0,90.0,97.0,87.0,96.0,91.0,86.0,91.0,...,85.0,59.0,94.0,48.0,22.0,Left,RF,Messi,Argentina,4.0
1,94,84.0,94.0,81.0,88.0,77.0,94.0,89.0,91.0,87.0,...,95.0,79.0,93.0,63.0,29.0,Right,ST,C. Ronaldo,Portugal,4.0
2,92,79.0,87.0,84.0,96.0,78.0,95.0,94.0,90.0,96.0,...,80.0,49.0,82.0,56.0,36.0,Right,LW,Neymar,Brazil,5.0
3,91,17.0,13.0,50.0,18.0,51.0,42.0,57.0,58.0,60.0,...,31.0,64.0,12.0,38.0,30.0,Right,GK,Lean,Spain,3.0
4,91,93.0,82.0,92.0,86.0,91.0,91.0,78.0,76.0,79.0,...,91.0,75.0,91.0,76.0,61.0,Right,RCM,Normal,Belgium,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,47,34.0,38.0,49.0,42.0,45.0,43.0,54.0,57.0,60.0,...,43.0,47.0,38.0,46.0,46.0,Right,CM,Lean,England,2.0
18203,47,23.0,52.0,43.0,39.0,25.0,40.0,41.0,39.0,38.0,...,41.0,67.0,42.0,47.0,16.0,Right,ST,Normal,Sweden,2.0
18204,47,25.0,40.0,38.0,45.0,28.0,44.0,70.0,69.0,50.0,...,45.0,32.0,45.0,32.0,15.0,Right,ST,Normal,England,3.0
18205,47,44.0,50.0,42.0,51.0,32.0,52.0,61.0,60.0,52.0,...,64.0,48.0,34.0,33.0,22.0,Right,RW,Lean,England,3.0


In [9]:
traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
features = traindf.columns

traindf = traindf.dropna()

In [10]:
traindf = pd.DataFrame(traindf,columns=features)

In [65]:
y = traindf['Overall']>=87
X = traindf.copy()
del X['Overall']

In [67]:
X.head()

,Crossing,Finishing,ShortPassing,Dribbling,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Stamina,...,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nationality_Venezuela,Nationality_Wales,Nationality_Zambia,Nationality_Zimbabwe
0,84.0,95.0,90.0,97.0,87.0,96.0,91.0,86.0,91.0,72.0,...,False,False,False,False,False,False,False,False,False,False
1,84.0,94.0,81.0,88.0,77.0,94.0,89.0,91.0,87.0,88.0,...,False,False,False,False,False,False,False,False,False,False
2,79.0,87.0,84.0,96.0,78.0,95.0,94.0,90.0,96.0,81.0,...,False,False,False,False,False,False,False,False,False,False
3,17.0,13.0,50.0,18.0,51.0,42.0,57.0,58.0,60.0,43.0,...,False,False,False,False,False,False,False,False,False,False
4,93.0,82.0,92.0,86.0,91.0,91.0,78.0,76.0,79.0,90.0,...,False,False,False,False,False,False,False,False,False,False


In [69]:
len(X.columns)

223

### Set some fixed set of features

In [72]:
feature_name = list(X.columns)
# no of maximum features we need to select
num_feats=30

## Filter Feature Selection - Pearson Correlation

### Pearson Correlation function

In [18]:
def cor_selector(X, y,num_feats):
    
    cor_list = []
    feature_name = X.columns.tolist()
    
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    
    cor_support = [True if i in cor_feature else False for i in feature_name]
    
    return cor_support, cor_feature

In [19]:
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


### List the selected features from Pearson Correlation

In [21]:
cor_feature

['Nationality_Costa Rica',
 'Position_LAM',
 'Nationality_Uruguay',
 'Acceleration',
 'SprintSpeed',
 'Strength',
 'Nationality_Gabon',
 'Nationality_Slovenia',
 'Stamina',
 'Weak Foot',
 'Agility',
 'Crossing',
 'Nationality_Belgium',
 'Dribbling',
 'ShotPower',
 'LongShots',
 'Finishing',
 'BallControl',
 'FKAccuracy',
 'LongPassing',
 'Volleys',
 'ShortPassing',
 'Position_RF',
 'Position_LF',
 'Body Type_Neymar',
 'Body Type_Messi',
 'Body Type_Courtois',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Body Type_C. Ronaldo',
 'Reactions']

## Filter Feature Selection - Chi-Sqaure

In [23]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

### Chi-Squared Selector function

In [25]:
def chi_squared_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    bestfeatures = SelectKBest(score_func=chi2, k=num_feats)
    topfeatures = bestfeatures.fit(X,y)
    chi_support = pd.DataFrame(topfeatures.scores_, columns=["Score"])
    chi_feature = pd.DataFrame(X.columns, columns=["Feature"])

    selected_mask = topfeatures.get_support()  
     
    chi_support = X.loc[:, selected_mask].values.tolist()
    chi_feature = X.columns[selected_mask].tolist()

    # Your code ends here
    return chi_support, chi_feature

In [26]:
chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
print(str(len(chi_feature)), 'selected features')

30 selected features


### List the selected features from Chi-Square 

In [28]:
chi_feature

['Crossing',
 'Finishing',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Stamina',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Position_LF',
 'Position_RF',
 'Body Type_C. Ronaldo',
 'Body Type_Courtois',
 'Body Type_Messi',
 'Body Type_Neymar',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Belgium',
 'Nationality_Gabon',
 'Nationality_Slovenia',
 'Nationality_Uruguay']

## Wrapper Feature Selection - Recursive Feature Elimination

In [30]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

### RFE Selector function

In [32]:
def rfe_selector(X, y, num_feats):
   
    lr = LogisticRegression(solver='lbfgs')
    rfe_lr = RFE(estimator=lr, 
             n_features_to_select=num_feats,
             step=1,
             verbose=5)
    rfe_lr = rfe_lr.fit(X, y)
    
    rfe_lr_support = rfe_lr.get_support()
    
    rfe_feature = X.columns[rfe_lr_support].tolist()
    rfe_support = X.loc[:, rfe_lr_support].values.tolist()
    
    return rfe_support, rfe_feature

In [33]:
rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 223 features.


C:\Users\wayne\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting estimator with 222 features.


C:\Users\wayne\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting estimator with 221 features.


...

Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.


C:\Users\wayne\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.


C:\Users\wayne\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
30 selected features


### List the selected features from RFE

In [35]:
rfe_feature

['Reactions',
 'Preferred Foot_Right',
 'Position_CAM',
 'Position_CF',
 'Position_CM',
 'Position_LAM',
 'Position_LCB',
 'Position_LW',
 'Position_RB',
 'Position_RF',
 'Position_RW',
 'Body Type_Courtois',
 'Body Type_Lean',
 'Nationality_Austria',
 'Nationality_Bosnia Herzegovina',
 'Nationality_Costa Rica',
 'Nationality_Croatia',
 'Nationality_Denmark',
 'Nationality_Gabon',
 'Nationality_Greece',
 'Nationality_Italy',
 'Nationality_Netherlands',
 'Nationality_Portugal',
 'Nationality_Senegal',
 'Nationality_Serbia',
 'Nationality_Slovakia',
 'Nationality_Slovenia',
 'Nationality_Switzerland',
 'Nationality_Uruguay',
 'Nationality_Wales']

## Embedded Selection - Lasso: SelectFromModel

In [37]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [38]:
def embedded_log_reg_selector(X, y, num_feats):
    
    logreg = LogisticRegression(penalty='l1', solver='liblinear')
    embedded_lr_selector = SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear', max_iter=50000), max_features=num_feats)
    embedded_lr_selector = embedded_lr_selector.fit(X, y)
    
    embedded_lr_support = embedded_lr_selector.get_support()
    embedded_lr_feature = X.loc[:, embedded_lr_support].columns.tolist()
    
    return embedded_lr_support, embedded_lr_feature

In [39]:
embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
print(str(len(embedded_lr_feature)), 'selected features')

30 selected features


In [40]:
embedded_lr_feature

['Crossing',
 'Finishing',
 'Dribbling',
 'LongPassing',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'Aggression',
 'Interceptions',
 'Weak Foot',
 'Preferred Foot_Right',
 'Position_CAM',
 'Position_CM',
 'Position_LCB',
 'Position_LM',
 'Position_LW',
 'Position_RB',
 'Position_RW',
 'Body Type_Lean',
 'Nationality_Brazil',
 'Nationality_Croatia',
 'Nationality_England',
 'Nationality_France',
 'Nationality_Germany',
 'Nationality_Italy',
 'Nationality_Netherlands',
 'Nationality_Uruguay']

## Tree based(Random Forest): SelectFromModel

In [42]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [43]:
def embedded_rf_selector(X, y, num_feats):
    rf = RandomForestClassifier(n_estimators=100)
    embedded_rf_selector = SelectFromModel(rf, 
                           max_features=num_feats)
    embedded_rf_selector = embedded_rf_selector.fit(X, y)
    embedded_rf_support = embedded_rf_selector.get_support()
    embedded_rf_feature = X.loc[:, embedded_rf_support].columns.tolist()
    
    return embedded_rf_support, embedded_rf_feature

In [44]:
embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
print(str(len(embedded_rf_feature)), 'selected features')

23 selected features


In [45]:
embedded_rf_feature

['Crossing',
 'Finishing',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Stamina',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Weak Foot',
 'Body Type_Courtois',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Germany']

## Tree based(Light GBM): SelectFromModel

In [47]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

In [48]:
def embedded_lgbm_selector(X, y, num_feats):

    lgbmc = LGBMClassifier(n_estimators=500,
                      learning_rate=0.05,
                      num_leaves=32,
                      colsample_bytree=0.2,
                      reg_alpha=3,
                      reg_lambda=1,
                      min_split_gain=0.01,
                      min_child_weight=40)
    embedded_lgbm_selector = SelectFromModel(lgbmc,
                                         max_features=num_feats)
    embedded_lgbm_selector = embedded_lgbm_selector.fit(X, y)
    embedded_lgbm_support = embedded_lgbm_selector.get_support()
    embedded_lgbm_feature = X.loc[:, embedded_lgbm_support].columns.tolist()
    return embedded_lgbm_support, embedded_lgbm_feature

In [49]:
embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
print(str(len(embedded_lgbm_feature)), 'selected features')

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 55, number of negative: 18104
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001518 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1812
[LightGBM] [Info] Number of data points in the train set: 18159, number of used features: 124
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003029 -> initscore=-5.796555
[LightGBM] [Info] Start training from score -5.796555
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[Light

In [50]:
embedded_lgbm_feature

['Crossing',
 'Finishing',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Stamina',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Weak Foot',
 'Preferred Foot_Left',
 'Preferred Foot_Right',
 'Position_CAM',
 'Position_CB',
 'Position_CDM',
 'Position_CF',
 'Position_CM',
 'Position_GK',
 'Position_LAM',
 'Position_LB']

## Putting all of it together: AutoFeatureSelector Tool

In [52]:
print(f"Length of feature_name: {len(feature_name)}")
print(f"Length of cor_support: {len(cor_support)}")
print(f"Length of chi_support: {len(chi_support)}")
print(f"Length of rfe_support: {len(rfe_support)}")
print(f"Length of embedded_lr_support: {len(embedded_lr_support)}")
print(f"Length of embedded_rf_support: {len(embedded_rf_support)}")
print(f"Length of embedded_lgbm_support: {len(embedded_lgbm_support)}")

Length of feature_name: 223
Length of cor_support: 223
Length of chi_support: 18159
Length of rfe_support: 18159
Length of embedded_lr_support: 223
Length of embedded_rf_support: 223
Length of embedded_lgbm_support: 223


In [53]:
pd.set_option('display.max_rows', None)

# 找出最大長度
max_length = max(len(feature_name), len(cor_support), len(chi_support),
                 len(rfe_support), len(embedded_lr_support), 
                 len(embedded_rf_support), len(embedded_lgbm_support))

# 將所有列補全到相同的長度
feature_name = feature_name + [""] * (max_length - len(feature_name))
cor_support = cor_support + [False] * (max_length - len(cor_support))
chi_support = chi_support + [False] * (max_length - len(chi_support))
rfe_support = rfe_support + [False] * (max_length - len(rfe_support))
embedded_lr_support = embedded_lr_support + [False] * (max_length - len(embedded_lr_support))
embedded_rf_support = embedded_rf_support + [False] * (max_length - len(embedded_rf_support))
embedded_lgbm_support = embedded_lgbm_support + [False] * (max_length - len(embedded_lgbm_support))


# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgbm_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

ValueError: operands could not be broadcast together with shapes (223,) (17936,) 

## Can you build a Python script that takes dataset and a list of different feature selection methods that you want to try and output the best (maximum votes) features from all methods?

In [59]:
def preprocess_dataset(dataset_path):
    # Load dataset
    data = pd.read_csv(dataset_path)

    # Assuming the last column is the target variable
    y = data.iloc[:, -1]
    X = data.iloc[:, :-1]

    num_feats = X.shape[1]
    
    return X, y, num_feats

In [61]:
def autoFeatureSelector(dataset_path, methods=[]):
    # Parameters
    # data - dataset to be analyzed (csv file)
    # methods - various feature selection methods we outlined before, use them all here (list)
    
    # preprocessing
    X, y, num_feats = preprocess_dataset(dataset_path)
    
    # Run every method we outlined above from the methods list and collect returned best features from every method
    if 'pearson' in methods:
        cor_support, cor_feature = cor_selector(X, y,num_feats)
    if 'chi-square' in methods:
        chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
    if 'rfe' in methods:
        rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
    if 'log-reg' in methods:
        embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
    if 'rf' in methods:
        embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
    if 'lgbm' in methods:
        embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
    
    
    # Combine all the above feature list and count the maximum set of features that got selected by all methods
    #### Your Code starts here (Multiple lines)
    # Combine all selected features into a single list
    all_features = []
    if 'pearson' in methods:
        all_features.extend(cor_feature)
    if 'chi-square' in methods:
        all_features.extend(chi_feature)
    if 'rfe' in methods:
        all_features.extend(rfe_feature)
    if 'log-reg' in methods:
        all_features.extend(embedded_lr_feature)
    if 'rf' in methods:
        all_features.extend(embedded_rf_feature)
    if 'lgbm' in methods:
        all_features.extend(embedded_lgbm_feature)
    
    # Count occurrences of each feature
    feature_votes = Counter(all_features)
    
    # Select features with maximum votes
    max_votes = max(feature_votes.values())
    best_features = [feature for feature, votes in feature_votes.items() if votes == max_votes]

    #### Your Code ends here
    return best_features

In [63]:
best_features = autoFeatureSelector(dataset_path="data/fifa19.csv", methods=['pearson', 'chi-square', 'rfe', 'log-reg', 'rf', 'lgbm'])
best_features

FileNotFoundError: [Errno 2] No such file or directory: 'data/fifa19.csv'

### Last, Can you turn this notebook into a python script, run it and submit the python (.py) file that takes dataset and list of methods as inputs and outputs the best features

In [ ]:
#Yes